In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [4]:
load_dotenv(override=True)

True

# Step 1: Agent Workflow

In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [8]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini"
)

In [14]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

Navigating the complexities of SOC 2 compliance can be a daunting challenge for many organizations. At ComplAI, we understand that maintaining compliance while maximizing operational efficiency is a top priority for businesses like yours.

Our AI-powered SaaS tool simplifies the SOC 2 compliance process by automating documentation, tracking compliance metrics, and preparing you for audits with ease. With ComplAI, you can streamline your compliance efforts, reduce audit preparation time, and ensure you meet industry standards without the usual headaches.

I would love to schedule a brief call to discuss how ComplAI can support your compliance journey and help you stay ahead of the curve. Are you available for a quick chat this week?

Thank you for considering ComplAI. I look forward to the possibility of working together.

Best regards,

[Your Name]  
[Your Position]  


In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")
